In [1]:
from gurobipy import *  # import the optimize solver Gurobi

In [2]:
number_of_week = 3 # Set the index for number of week(1, 2, 3)
m = Model() # Import and create the model

Using license file C:\Users\chaoz\gurobi_licence\gurobi.lic
Academic license - for non-commercial use only


In [3]:
# Set the input Parameter: 
unit_production_cost = [130, 140, 150]  # Production cost of per unit in each week  
unit_holding_cost = 20  # Storage/Holding cost incurred per unit  
demand = [2000, 1000, 1500]   # Demand in each week 
initial_inventory = 500  # Inventory at the beginning of week 1
a = 0.5 # Fraction of the goods produced during a week that can be used to meet the current week’s demand

In [4]:
# Set the Variable list: Units of Switch to be ordered in each week
# Set the variable nx to integer number
nx = [] 
for i in range(number_of_week):
    nx.append(m.addVar(vtype=GRB.INTEGER, name='nx{}'.format(i + 1)))

# Caculate inventory at the end of each month
# ni = []
# ni.append(initial_inventory)
# for i in range(len(nx)):
#     xx = nx[i] + ni[i] - demand[i]
#     ni.append(xx)

ni = []
for i in range(len(nx)):
    if i <= 0:
        xx = nx[i] + initial_inventory  - demand[i]
    else:
        xx = nx[i] + ni[i - 1] - demand[i]
    ni.append(xx)

In [5]:
# Set the Minimize Obijective: Total Cost
m.setObjective(quicksum([unit_production_cost[i]*nx[i] for i in range(len(nx))]) + quicksum([unit_holding_cost*ni[i] for i in range(len(nx))]),  GRB.MINIMIZE)

In [6]:
# Set Non Negative decision variable
c1 = []
for i in range(len(nx)):
    c1.append(m.addConstr(nx[i] >= 0))
    
# Demand must be satisfied for each week 
c2 = []
for i in range(len(nx)):
    if i > 0:
        c2.append(m.addConstr(ni[i-1] + a*nx[i] >= demand[i]))
    else:
        c2.append(m.addConstr(initial_inventory + a*nx[i] >= demand[i]))

In [7]:
# Run the optimize solver
m.optimize()

Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 6 rows, 3 columns and 9 nonzeros
Model fingerprint: 0x0028c51c
Variable types: 0 continuous, 3 integer (0 binary)
Coefficient statistics:
  Matrix range     [5e-01, 1e+00]
  Objective range  [2e+02, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+03, 4e+03]
Found heuristic solution: objective 750000.00000
Presolve removed 6 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.03 seconds
Thread count was 1 (of 8 available processors)

Solution count 2: 590000 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.900000000000e+05, best bound 5.900000000000e+05, gap 0.0000%


In [8]:
# Get the Optimal Solution for X
m.printAttr('X')


    Variable            X 
-------------------------
         nx1         3000 
         nx2         1000 


In [9]:
# Get the Optimal Objective Value
m.ObjVal

590000.0

In [10]:
# Get the optimal solution list
m.X

[3000.0, 1000.0, 0.0]

In [11]:
# Get the end of inventory for each week
nin = []
initial_inventory = 500
for i in range(3):
    if i > 0:
        initial_inventory = nin[i-1]
    xx = m.X[i] + initial_inventory - demand[i]
    nin.append(xx)
print(nin)

[1500.0, 1500.0, 0.0]
